In [1]:
import re

In [12]:
string = '((((A->B)|C)->D)&F)'

In [3]:
def contains(a,b):
    a = list(a)
    b = list(b)
    result = True
    for element in a:
        result = result and (element in b)
    return result

In [4]:
def evaluate(sentence, model):
    keys = model.keys()
    
    #put variables values
    for key in keys:
        parts = re.search('[~\(\&\|\-\>](' + key + ')[\)\&\|\-\<]', sentence)
        while (parts != None):
            part = parts.group(0)
            part = part[1:-1]
            sentence = sentence.replace(part, model[key])
            parts = re.search('[~\(\&\|\-\>](' + key + ')[\)\&\|\-\<]', sentence)
            
    #replace values with not ~
    not_replacement = {"~0" : "1", "~1" : "0"}
    for val in not_replacement.keys():
        parts = re.search('[\(\&\|\-\>]' + val + '[\)\&\|\-\<]', sentence)
        while parts != None:
            part = parts.group(0)
            part = part[1:-1]
            sentence = sentence.replace(part, not_replacement[val])
            parts = re.search('[\(\&\|\-\>]' + val + '[\)\&\|\-\<]', sentence)
    
    #compute sentence value
    parts = re.search('(\([^\(\)]*\))', sentence)
    while parts != None:
        part = parts.group(0)
        sentence = sentence.replace(part, evaluate_simple(part))
        parts = re.search('(\([^\(\)]*\))', sentence)
    return bool(int(sentence))

In [5]:
def evaluate_simple(sentence):
    if sentence[0] == '(' and sentence[-1] == ')':
        sentence = sentence[1:-1]
    
    #eval and = &
    if sentence == '0&0' or sentence== '0&1' or sentence == '1&0':
        return '0'
    else:
        if sentence == '1&1':
            return '1'
    
    #eval or = |
    if sentence == '1|1' or sentence== '0|1' or sentence == '1|0':
        return '1'
    else:
        if sentence == '0|0':
            return '0'
    
    #eval implication = ->
    if sentence == '0->0' or sentence== '0->1' or sentence == '1->1':
        return '1'
    else:
        if sentence == '1->0':
            return '0'
    
    #eval implication = <-
    if sentence == '0<-0' or sentence== '1<-0' or sentence == '1<-1':
        return '1'
    else:
        if sentence == '0<-1':
            return '0'
    
    #eval equivalence = <->
    if sentence == '0<->0' or sentence == '1<->1':
        return '1'
    else:
        if sentence == '1<->0' or sentence == '0<->1':
            return '0'
    
    return sentence

In [6]:
def evaluate_simple_test():
    val = ["0", "1"]
    operators = ['&', '|', '->', '<-', '<->']
    for op in operators:
        for v1 in val:
            for v2 in val:
                sentence = '(' + v1 + op + v2 + ')'
                res = evaluate_simple(sentence)
                print sentence + "   " + res
                sentence = v1 + op + v2
                res = evaluate_simple(sentence)
                print sentence + "   " + res

In [30]:
evaluate(string, model)

False

In [26]:
model = {"A" : "0", "B" : "1", "C" : "0", "D" : "0", "F" : "1"}

In [18]:
evaluate_simple('(0&0)')

'0'

In [19]:
def extract_symbols(sentence):
    result = []
    symbols = re.search('[\(\&\|\>\-\~][^\(\)\&\|\<\>\-\~]+[\)\&\|\<\-]', sentence)
    while (symbols != None):
        symbol = symbols.group(0)
        symbol = symbol[1:-1]
        result.append(symbol)
        sentence = sentence.replace(symbol, '')
        symbols = re.search('[\(\&\|\>\-\~][^\(\)\&\|\<\>\-\~]+[\)\&\|\<\-]', sentence)
    return result

In [20]:
def tt_entails(KB, sentence):
    #input: KB is list of sentences(str)
    #       sentence is a str
    
    #find all symbols
    symbols = extract_symbols(sentence)
    for s in KB:
        symbols += extract_symbols(s)
    symbols = list(set(symbols))
    
    return tt_check_all(KB, sentence, symbols, {})

In [21]:
def satisfied(KB, model):
    if type(KB) == list:
        return satisfied_kb(KB, model)
    if type(KB) == str:
        return evaluate(KB, model)

def satisfied_kb(KB, model):
    symbols = []
    for s in KB:
        symbols += extract_symbols(s)
    symbols = set(symbols)
    
    if contains(symbols, set(model.keys())):
        result = True
        for s in KB:
            result = result and evaluate(s, model)
        return result
    else:
        return False

In [22]:
def tt_check_all(KB, sentence, symbols, model):
    if len(symbols) == 0:
        if satisfied(KB, model):
            return satisfied(sentence, model)
        else:
            return True
    else:
        result = True
        model[symbols[0]] = '0'
        result = result and tt_check_all(KB, sentence, symbols[1:], model)
        model[symbols[0]] = '1'
        result = result and tt_check_all(KB, sentence, symbols[1:], model)
        return result

In [23]:
KB = ['(A|B)', '(B&F)', '(C->F)']

In [32]:
satisfied(KB,model)

True

In [33]:
model

{'A': '0', 'B': '1', 'C': '0', 'D': '0', 'F': '1'}

In [115]:
del(model['G'])

In [34]:
KB1 = ['(P)', '(P->Q)']
sentence1 = '(Q)'

In [35]:
tt_entails(KB1, sentence1)

True

In [36]:
evaluate('(Q)',{'Q' : '1'})

True

In [37]:
evaluate_simple('(1)')

'1'